In [1]:
%%html
<style>
.container{width:85%}
<style>

In [ ]:
!conda install -c frankong sigpy

In [2]:
import RL_samp
from RL_samp.header import *
from RL_samp.utils import *
from RL_samp.replay_buffer import *
from RL_samp.models import poly_net, val_net
# from RL_samp.reconstructors import sigpy_solver
from RL_samp.policies import DQN
from RL_samp.trainers import DeepQL_trainer #, AC1_ET_trainer

from importlib import reload
import matplotlib.pyplot as plt
from matplotlib import colors
import torch.nn.functional as Func

def rolling_mean(x,window):
    window = int(window)

    # Stephen: for large data, the above gets a bit slow, so we can do this:
#   y = np.convolve(x, np.ones(window)/window, mode='valid')
#   return y
    # or https://stackoverflow.com/a/27681394
    cumsum = np.cumsum(np.insert(x, 0, 0))
    return (cumsum[window:] - cumsum[:-window]) / float(window)

In [ ]:
fs_dataloader = dataloader()

In [ ]:
fs_dataloader.load()

In [ ]:
train_percentage = 0.75

datapath = '/home/ec2-user/SageMaker/data/OCMR_fully_sampled_images/'
ncfiles = list([])
for file in os.listdir(datapath):
    if file.endswith(".pt") and file.startswith('fs'):
        ncfiles.append(file)
print('debug: ncfiles[0] is ', ncfiles[0])
print('Number of useful files: ', len(ncfiles))
random.shuffle(ncfiles)
print('debug: ncfiles[0] is ', ncfiles[0])
train_file_num = int(np.ceil(train_percentage*len(ncfiles)))

train_files = ncfiles[0:train_file_num]
print('Number of Train  files: ', len(train_files))
test_files = ncfiles[train_file_num:]
print('Number of Test   files: ', len(test_files))

In [ ]:
np.savez('/home/ec2-user/SageMaker/RLsamp/train_files.npz',files=train_files)
np.savez('/home/ec2-user/SageMaker/RLsamp/test_files.npz', files=test_files)

In [ ]:
datapath = '/home/ec2-user/SageMaker/data/OCMR_fully_sampled_images/'
ncfiles = list([])
for file in os.listdir(datapath):
    if file.endswith(".pt") and file.startswith('fs'):
        ncfiles.append(file)

In [ ]:
useful_files = np.load('/home/ec2-user/SageMaker/RLsamp/useful_files.npz', allow_pickle=True)['files']
train_files  = np.load('/home/ec2-user/SageMaker/RLsamp/train_files.npz', allow_pickle=True)['files']
test_files   = np.load('/home/ec2-user/SageMaker/RLsamp/test_files.npz', allow_pickle=True)['files']

In [ ]:
time_dim = [[],[]]
slice_dim = [[],[]]
rep_dim = [[],[]]
for row in useful_files:
    if row[0].startswith('fs'):
        if (row[0][:-3] + '.pt') in train_files:
            time_dim[0].append(row[1][4])
            slice_dim[0].append(row[1][6])
            rep_dim[0].append(row[1][7])
        if (row[0][:-3] + '.pt') in test_files:
            time_dim[1].append(row[1][4])
            slice_dim[1].append(row[1][6])
            rep_dim[1].append(row[1][7])

In [ ]:
sorted(rep_dim[1])

In [ ]:
print('train: ',train_files, len(train_files) )
print('test: ', test_files, len(test_files) )


In [ ]:
ex_data = torch.load('/home/ec2-user/SageMaker/data/OCMR_fully_sampled_images/fs_0074_1_5T.h5')

In [ ]:
loader.reset()
iterMax = 2000
for ind in range(iterMax):
    loader.test()

In [ ]:
### DQN Parameter settings

## image parameters
heg = 192
wid = 144

## reconstructor parameters
max_iter = 50
L = 5e-3
solver = 'ADMM'

## trainer parameters
discount    = .5
memory_len  = 20
t_backtrack = 3
base        = 5
budget      = 13
episodes    = 1
save_freq   = 10
batch_size  = 2
ngpu        = 1
lr          = 1e-3
eps         = 1e-3
double_q    = False

In [ ]:
reload(RL_samp)
import RL_samp
import RL_samp.trainers
from RL_samp.trainers import DeepQL_trainer


In [ ]:
loader  = ocmrLoader(ncfiles,batch_size=1,t_backtrack=t_backtrack)
memory  = ReplayMemory(capacity=memory_len,
                       curr_obs_shape=(t_backtrack,heg,wid),
                       mask_shape=(wid),
                       next_obs_shape=(1,heg,wid),
                       batch_size=batch_size,
                       burn_in=batch_size)
model   = poly_net(samp_dim=wid)
policy  = DQN(model,memory,max_iter=max_iter,ngpu=ngpu,gamma=discount,lr=lr,double_q_mode=double_q,
              solver=solver,max_iter=max_iter,L=L)
trainer = DeepQL_trainer(loader,policy,episodes=episodes,
                         eps=eps,
                         base=base,budget=budget,
                         ngpu=ngpu)
trainer.train()

In [ ]:
### AC1 Parameter settings

## image parameters
heg = 192
wid = 144

## reconstructor parameters
max_iter = 50
L = 5e-3
solver = 'ADMM'

## trainer parameters
discount    = .9
t_backtrack = 3
base        = 5
budget      = 13
episodes    = 1
save_freq   = 10
batch_size  = 2
ngpu        = 0
lr          = 1e-3
eps         = 1e-3
double_q    = False

In [ ]:
reload(RL_samp)
import RL_samp
from RL_samp.trainers import AC1_trainer

In [ ]:
loader  = ocmrLoader(ncfiles,batch_size=1,t_backtrack=t_backtrack)
p_net   = poly_net(samp_dim=wid,softmax=True)
v_net   = val_net()
trainer = AC1_trainer(loader, polynet=p_net, valnet=v_net,
                      base=base, budget=budget,
                      gamma=discount,
                      lr=lr,
                      solver=solver, max_iter=max_iter, L=L,
                      ngpu=ngpu)
trainer.run()

In [ ]:
loader  = ocmrLoader(ncfiles,batch_size=1,t_backtrack=t_backtrack)
p_net   = poly_net(samp_dim=wid,softmax=True)
v_net   = val_net(slope=.5,scale=10)
trainer = AC1_ET_trainer(loader, polynet=p_net, valnet=v_net,
                      base=base, budget=budget,
                      gamma=discount,
                      solver=solver, max_iter=max_iter, L=L, reward_scale=9e2,
                      ngpu=ngpu)

In [ ]:
trainer.run()

## view training history: DDQN

In [ ]:
hist_dir = '/home/ec2-user/SageMaker/RLsamp/output/DQN_doubleQ_True_ba8_bu16_hist_2023-04-10_BA8BU16_LF_0G_600Epochs.pt'
data = torch.load(hist_dir)
print(data.keys())

In [ ]:
## Loss Convergence Plot
plt.rcParams['text.usetex'] = True
plt.clf()
plt.figure()
window_small = 400
num_ticks = 6
epochs = 600
sequence_length = len(rolling_mean(data['training_record']['rmse'],window_small))

plt.plot(rolling_mean(data['training_record']['rmse'],window_small), linewidth=0.8, label='DDQN')
plt.plot(rolling_mean(data['training_record']['rmse_rand'],window_small), linestyle=':', linewidth=.5,  label='rand.')
plt.plot(rolling_mean(data['training_record']['rmse_lowfreq'],window_small), linestyle='--', linewidth=.5,  label='lowfreq.')
plt.xticks(ticks=np.arange(0, sequence_length, sequence_length//num_ticks), labels=[epochs//num_ticks * i for i in range(0,num_ticks+1)],fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Epochs', fontsize=20)
plt.ylabel(r'Normalized $\ell_2$ error', fontsize=20)
plt.legend(loc='best',fontsize=13)
# figname = './figures/DDQN_RMSE_convergence_plot.pdf'
# plt.savefig(figname, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
## make mask convergence plots
sampdim = 144
TOTAL_PTS = 700

T  = len(data['training_record']['recon_samples'])
t0 = 0
t1 = 0

Ts1 = np.arange(t0,t1,1) 
Ts2 = np.linspace(t1, T-1, TOTAL_PTS - (t1-t0)).astype(int)
Ts  = list(Ts1) + list(Ts2)
series = [data['training_record']['recon_samples'][t] for t in Ts]
masks = [item[2] for item in series]
mask_mat = torch.zeros(sampdim,len(Ts))
for ind in range(len(Ts)):
    mask_mat[:,ind] = F.fftshift(masks[ind])

In [ ]:
plt.clf()
plt.figure(figsize=(20,3))
label_fontsize = 20
plt.imshow(mask_mat,origin='lower',cmap='Greys')
# plt.xlabel('time',fontsize=label_fontsize)
# plt.ylabel('frequencies',fontsize=label_fontsize)
plt.yticks([],[])

if t1 > t0:
    plt.xticks([len(Ts1)],[r'$t_\ast$'],fontsize=15)
else:
    plt.xticks([],[])
    
figname = '/home/ec2-user/SageMaker/RLsamp/figures/DDQN_mask.pdf'
plt.savefig(figname, bbox_inches='tight', 
               transparent=True)
plt.show()

In [16]:
# hist_dir = '/home/ec2-user/SageMaker/RLsamp/output/EVAL_DQN_doubleQ_True_ba8_bu16_2023-05-02.pt'
hist_dir = '/home/ec2-user/SageMaker/RLsamp/output/EVAL_DQN_doubleQ_True_ba8_bu16_2023-05-08.pt'
data = torch.load(hist_dir)
print(data.keys())
print(data['testing_record'].keys())

dict_keys(['testing_record'])
dict_keys(['rmse', 'rmse_rand', 'rmse_lowfreq', 'ssim', 'ssim_rand', 'ssim_lowfreq', 'recon_samples', 'recon_samples_rand'])


In [17]:
print('DDQN rmse test: ', np.mean(data['testing_record']['rmse']))
print('Rand. rmse test    : ', np.mean(data['testing_record']['rmse_rand']))
print('LowFreq. rmse test : ', np.mean(data['testing_record']['rmse_lowfreq']))
print('\n')
print('DDQN rmse test: ', np.mean(data['testing_record']['ssim']))
print('Rand. rmse test    : ', np.mean(data['testing_record']['ssim_rand']))
print('LowFreq. rmse test : ', np.mean(data['testing_record']['ssim_lowfreq']))

DDQN rmse test:  0.53779761676258
Rand. rmse test    :  0.679948924661155
LowFreq. rmse test :  0.53779761676258


DDQN rmse test:  0.6385296720234377
Rand. rmse test    :  0.4177503257576175
LowFreq. rmse test :  0.6385296720234377


In [ ]:
T = 22

series = data['testing_record']['recon_samples'][0:T]
gts    = [item[0].squeeze().cpu() for item in series]
recons = [item[1].squeeze().cpu() for item in series]

figsize = (10,40)
nrows = 6
ncols = 3
fig, axs = plt.subplots(nrows=nrows,ncols=ncols,figsize=figsize)

window_large = 300
window_small = 300

frame = np.linspace(0,T-1,nrows).astype(int)
for i in range(nrows):
    axs[i,0].imshow(gts[frame[i]],vmin=0,vmax=1,cmap='gray')
    axs[i,1].set_title(fr'$t={frame[i]+1}$')
    axs[i,1].imshow(recons[frame[i]],vmin=0,vmax=1,cmap='gray')
    hdl = axs[i,2].imshow(abs(recons[frame[i]]-gts[frame[i]])/gts[frame[i]] , norm=colors.LogNorm(vmin=0.1,vmax=10), cmap='winter')
    fig.colorbar(hdl, ax=axs[i,2], location='right', anchor=(0, 0.5), shrink=0.5)

for i in range(nrows):
    for j in range(ncols):
        axs[i,j].set_xticks([])
        axs[i,j].set_yticks([])
        
plt.subplots_adjust(wspace=.1, hspace=0)
plt.show()

In [ ]:
hist['horizon_rewards'] = [item.item() for item in hist['horizon_rewards']]

In [ ]:
figsize = (12,13)
keys = list(data['training_record'].keys())
fig, axs = plt.subplots(nrows=3,ncols=3,figsize=figsize)

window_large = 500
window_small = 300

axs[0,0].plot(rolling_mean(data['training_record']['loss'], window_large))
axs[0,0].set_title('running loss')

axs[0,1].plot(rolling_mean(data['training_record']['rmse'],window_small), label='RL')
axs[0,1].plot(rolling_mean(data['training_record']['rmse_rand'],window_small), label='rand')
axs[0,1].plot(rolling_mean(data['training_record']['rmse_lowfreq'],window_small), label='lowfreq.')
axs[0,1].set_title('RMSE - recon')
axs[0,1].legend(loc='best')
# axs[0,1].set_yscale('log')

axs[0,2].plot(rolling_mean(data['training_record']['horizon_rewards'],window_small))
axs[0,2].set_title('Horizon Reward')
# axs[0,2].set_yscale('log')

axs[1,0].plot(rolling_mean(data['training_record']['q_values_mean'],window_large))
axs[1,0].set_title('Q values mean')


axs[1,1].plot(rolling_mean(data['training_record']['grad_norm'],window_large))
axs[1,1].set_title('Grad Norm')
axs[1,1].set_yscale('log')

axs[1,2].axis('off')

gt = axs[2,0].imshow(data['training_record']['recon_samples'][-1][0][0,0,:,:].cpu(), origin='lower')
axs[2,0].set_title('Target gt')
fig.colorbar(gt, ax=axs[2,0], location='right', anchor=(0, 0.3), shrink=0.7)

recon = axs[2,1].imshow(data['training_record']['recon_samples'][-1][1][0,0,:,:].cpu(), origin='lower')
axs[2,1].set_title('Recon')
fig.colorbar(recon, ax=axs[2,1], location='right', anchor=(0, 0.3), shrink=0.7)

rel_error = axs[2,2].imshow((data['training_record']['recon_samples'][-1][1][0,0,:,:].cpu() - data['training_record']['recon_samples'][-1][0][0,0,:,:].cpu())/data['training_record']['recon_samples'][-1][0][0,0,:,:].cpu(),
                            origin='lower', norm=colors.LogNorm())
axs[2,2].set_title('Relative Error')
fig.colorbar(rel_error, ax=axs[2,2], location='right', anchor=(0, 0.3), shrink=0.7)


plt.show()

In [ ]:
print(keys)

## view training history: Actor-Critic

In [ ]:
hist_dir = '/home/ec2-user/SageMaker/RLsamp/output/AC1_ET_hist_base8_budget16_2023-04-14_tb8_BA8_BU16_200G_600E_magweg5_rwd1_val_1.pt'
data = torch.load(hist_dir)
print(data.keys())
print(data['training_record'].keys())

In [ ]:
figsize = (15,15)
fig, axs = plt.subplots(nrows=3,ncols=3,figsize=figsize)

window_large = 300
window_small = 300

axs[0,0].plot(rolling_mean(data['training_record']['poly_loss'], window_large))
axs[0,0].set_title('policy network running loss')

axs[0,1].plot(rolling_mean(data['training_record']['val_loss'],window_small))
axs[0,1].set_title('Value network running loss')
# axs[0,2].set_yscale('log')

axs[0,2].plot(rolling_mean(data['training_record']['v'],window_small))
axs[0,2].set_title('running v value')

axs[1,0].plot(rolling_mean(data['training_record']['horizon_rewards'],window_large))
axs[1,0].set_title('Horizon Rewards')


axs[1,1].plot(rolling_mean(data['training_record']['poly_grad_norm'],window_large))
axs[1,1].set_title('polynet Grad Norm')
# axs[1,1].set_yscale('log')

axs[1,2].plot(rolling_mean(data['training_record']['val_grad_norm'],window_large))
axs[1,2].set_title('valnet Grad Norm')
axs[1,2].set_yscale('log')

axs[2,0].plot(rolling_mean(data['training_record']['rmse'],window_small), label='RL')
axs[2,0].plot(rolling_mean(data['training_record']['rmse_rand'],window_small), label='rand')
axs[2,0].plot(rolling_mean(data['training_record']['rmse_lowfreq'],window_small), label='lowfreq.')
axs[2,0].set_title('RMSE - recon')
axs[2,0].legend(loc='best')
# axs[0,1].set_yscale('log')

gt = axs[2,1].imshow(data['training_record']['recon_samples'][-1][0][0,0,:,:])
axs[2,1].set_title('Recon')
fig.colorbar(gt, ax=axs[2,1], location='right', anchor=(0, 0.3), shrink=0.7)

recon = axs[2,2].imshow(data['training_record']['recon_samples'][-1][1][0,0,:,:])
axs[2,2].set_title('Target gt')
fig.colorbar(recon, ax=axs[2,2], location='right', anchor=(0, 0.3), shrink=0.7)


plt.show()

## view training history: REINFORCE

In [ ]:
## view training history: DDQN
hist_dir = '/home/ec2-user/SageMaker/RLsamp/output/REINFORCE_hist_2023-04-24_base8_budget16_BA8_BU16_E1000_G0_H1e-2_wTrue_magweg5_rwd1.pt'
data = torch.load(hist_dir)
print(data.keys())
print(data['training_record'].keys())

In [ ]:
## Loss Convergence Plot
plt.rcParams['text.usetex'] = True
plt.clf()
plt.figure()
window_small = 400
num_ticks = 6

epochs_total = 1000
epochs = 600
sequence_length = len(rolling_mean(data['training_record']['rmse'],window_small))
plot_range = epochs * sequence_length//epochs_total

plt.plot(rolling_mean(data['training_record']['rmse'][:plot_range],window_small), linewidth=0.8, label='REINFORCE')
plt.plot(rolling_mean(data['training_record']['rmse_rand'][:plot_range],window_small), linestyle=':', linewidth=.5,  label='rand.')
plt.plot(rolling_mean(data['training_record']['rmse_lowfreq'][:plot_range],window_small), linestyle='--', linewidth=.5,  label='lowfreq.')
plt.xticks(ticks=np.arange(0, plot_range, plot_range//num_ticks), labels=[epochs//num_ticks * i for i in range(0,num_ticks+1)],fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Epochs', fontsize=20)
plt.ylabel(r'Normalized $\ell_2$ error', fontsize=20)
plt.legend(loc='best',fontsize=13)
figname = './figures/REINFORCE_RMSE_convergence_plot.pdf'
plt.savefig(figname, bbox_inches='tight', 
               transparent=True)
plt.show()

In [ ]:
## make mask convergence plots
sampdim = 144
TOTAL_PTS = 700

T  = len(data['training_record']['recon_samples'])
t0 = 0
t1 = 0

Ts1 = np.arange(t0,t1,1) 
Ts2 = np.linspace(t1, T-1, TOTAL_PTS - (t1-t0)).astype(int)
Ts  = list(Ts1) + list(Ts2)
series = [data['training_record']['recon_samples'][t] for t in Ts]
masks = [item[2] for item in series]
mask_mat = torch.zeros(sampdim,len(Ts))
for ind in range(len(Ts)):
    mask_mat[:,ind] = F.fftshift(masks[ind])

In [ ]:
plt.clf()
plt.figure(figsize=(20,3))
label_fontsize = 20
plt.imshow(mask_mat,origin='lower',cmap='Greys')
# plt.xlabel('time',fontsize=label_fontsize)
# plt.ylabel('frequencies',fontsize=label_fontsize)
plt.yticks([],[])

if t1 > t0:
    plt.xticks([len(Ts1)],[r'$t_\ast$'],fontsize=15)
else:
    plt.xticks([],[])
    
figname = '/home/ec2-user/SageMaker/RLsamp/figures/REINFORCE_mask.pdf'
plt.savefig(figname, bbox_inches='tight', 
               transparent=True)
plt.show()

In [13]:
# hist_dir = '/home/ec2-user/SageMaker/RLsamp/output/Test_REINFORCE_hist_2023-05-02_base8_budget16_REINFORCE_test_magweg1.0_rwd1.pt'
hist_dir = '/home/ec2-user/SageMaker/RLsamp/output/Test_REINFORCE_hist_2023-05-08_base8_budget16_REINFORCE_test_magweg1.0_rwd1.pt'
data = torch.load(hist_dir)
print(data.keys())
print(data['testing_record'].keys())

dict_keys(['testing_record'])
dict_keys(['rmse', 'recon_samples', 'rmse_rand', 'rmse_lowfreq', 'ssim', 'ssim_rand', 'ssim_lowfreq'])


In [15]:
print('REINFORCE rmse test: ', np.mean(data['testing_record']['rmse']))
print('Rand. rmse test    : ', np.mean(data['testing_record']['rmse_rand']))
print('LowFreq. rmse test : ', np.mean(data['testing_record']['rmse_lowfreq']))
print('\n')
print('REINFORCE ssim test: ', np.mean(data['testing_record']['ssim']))
print('Rand. ssim test    : ', np.mean(data['testing_record']['ssim_rand']))
print('LowFreq. ssim test : ', np.mean(data['testing_record']['ssim_lowfreq']))

REINFORCE rmse test:  0.5352999488658667
Rand. rmse test    :  0.6796782487106651
LowFreq. rmse test :  0.53779761676258


REINFORCE ssim test:  0.6240303514238591
Rand. ssim test    :  0.42094609634867297
LowFreq. ssim test :  0.6385296720234378


In [ ]:
T = 16

series = data['testing_record']['recon_samples'][0:T]
gts    = [item[0].squeeze().cpu() for item in series]
recons = [item[1].squeeze().cpu() for item in series]

figsize = (10,25)
nrows = 5
ncols = 3
fig, axs = plt.subplots(nrows=nrows,ncols=ncols,figsize=figsize)

window_large = 300
window_small = 300

frame = np.linspace(0,T-1,nrows).astype(int)
for i in range(nrows):
    axs[i,0].imshow(gts[frame[i]],vmin=0,vmax=1,cmap='gray')
    axs[i,1].set_title(fr'$t={frame[i]+1}$')
    axs[i,1].imshow(recons[frame[i]],vmin=0,vmax=1,cmap='gray')
    hdl = axs[i,2].imshow(abs(recons[frame[i]]-gts[frame[i]])/gts[frame[i]] , norm=colors.LogNorm(vmin=0.1,vmax=10), cmap='winter')
    fig.colorbar(hdl, ax=axs[i,2], location='right', anchor=(0, 0.5), shrink=0.5)

for i in range(nrows):
    for j in range(ncols):
        axs[i,j].set_xticks([])
        axs[i,j].set_yticks([])

figname = '/home/ec2-user/SageMaker/RLsamp/figures/REINFORCE_recon_example.pdf'
plt.savefig(figname, bbox_inches='tight', 
               transparent=True)

plt.subplots_adjust(wspace=.05, hspace=0)
plt.show()

In [ ]:
ncfiles = np.load('/home/ec2-user/SageMaker/RLsamp/test_files.npz')['files']
t_backtrack = 8
datapath = '/home/ec2-user/SageMaker/data/OCMR_fully_sampled_images/'
loader  = ocmrLoader(ncfiles,batch_size=1,datapath=datapath,t_backtrack=t_backtrack,train_mode=False)


In [ ]:
loader.reset()

In [ ]:
file = 'fs_0021_3T.pt'  # pick an example file
data = torch.load(datapath+file)
print(data.shape)

In [ ]:
# Show the reconstructed cine image
from IPython.display import clear_output
import time

slc_idx = 0 # math.floor(SLC/2)
print(slc_idx)
for rep in range(5): # repeat the movie for 5 times
    for frame in range(data.shape[3]):
        clear_output(wait=True)
        plt.imshow(data[:,:,0,frame,0,slc_idx,:], vmin=0, vmax=data.max(), cmap = 'gray'); plt.axis('off');
        plt.colorbar()
        plt.show() 
        time.sleep(0.03)

In [ ]:
figsize = (15,15)
fig, axs = plt.subplots(nrows=3,ncols=3,figsize=figsize)

window_large = 300
window_small = 300

axs[0,0].plot(rolling_mean(data['training_record']['poly_loss'], window_large))
axs[0,0].set_title('policy network running loss')

axs[0,1].plot(rolling_mean(data['training_record']['poly_grad_norm'],window_small))
axs[0,1].set_title('policy network gradient norm')
axs[0,2].set_yscale('log')

axs[0,2].plot(rolling_mean(data['training_record']['action_prob'],window_small))
axs[0,2].set_title('action probability')

axs[1,0].plot(rolling_mean(data['training_record']['horizon_rewards'],window_large))
axs[1,0].set_title('Horizon Rewards')


# axs[1,1].plot(rolling_mean(data['training_record']['poly_grad_norm'],window_large))
# axs[1,1].set_title('polynet Grad Norm')
# # axs[1,1].set_yscale('log')

# axs[1,2].plot(rolling_mean(data['training_record']['val_grad_norm'],window_large))
# axs[1,2].set_title('valnet Grad Norm')
# axs[1,2].set_yscale('log')

axs[1,1].plot(rolling_mean(data['training_record']['rmse'],window_small), label='RL')
axs[1,1].plot(rolling_mean(data['training_record']['rmse_rand'],window_small), label='rand')
axs[1,1].plot(rolling_mean(data['training_record']['rmse_lowfreq'],window_small), label='lowfreq.')
axs[1,1].set_title('RMSE - recon')
axs[1,1].legend(loc='best')
# axs[0,1].set_yscale('log')

axs[1,2].axis('off')

gt = axs[2,0].imshow(data['training_record']['recon_samples'][-1][0][0,0,:,:], origin='lower')
axs[2,0].set_title('Target gt')
fig.colorbar(gt, ax=axs[2,0], location='right', anchor=(0, 0.3), shrink=0.7)

recon = axs[2,1].imshow(data['training_record']['recon_samples'][-1][1][0,0,:,:], origin='lower')
axs[2,1].set_title('Recon')
fig.colorbar(recon, ax=axs[2,1], location='right', anchor=(0, 0.3), shrink=0.7)

rel_error = axs[2,2].imshow((data['training_record']['recon_samples'][-1][1][0,0,:,:] - data['training_record']['recon_samples'][-1][0][0,0,:,:])/data['training_record']['recon_samples'][-1][0][0,0,:,:],
                            origin='lower', norm=colors.LogNorm())
axs[2,2].set_title('Relative Error')
fig.colorbar(rel_error, ax=axs[2,2], location='right', anchor=(0, 0.3), shrink=0.7)

plt.show()

# Run Tester : DDQN

In [ ]:
hist_dir = '/home/ec2-user/SageMaker/RLsamp/output/DQN_doubleQ_True_ba8_bu16_hist_2023-04-10_BA8BU16_LF_0G_600Epochs.pt'
hist = torch.load(hist_dir)

In [ ]:
from RL_samp.trainers import DeepQL_tester
from unet.unet_model_fbr import Unet

In [ ]:
# dataloader params
t_backtrack = 8
datapath = '/home/ec2-user/SageMaker/data/OCMR_fully_sampled_images/'

# unet params
unet_rand_dir = '/home/ec2-user/SageMaker/RLsamp/output/recon_models/unet_lowfreq_rand_0.0_fbr_2_chans_64base8_budget16.pt'
unet_lowfreq_dir = '/home/ec2-user/SageMaker/RLsamp/output/recon_models/unet_lowfreq_rand_1.0_fbr_2_chans_64base8_budget16.pt'
in_chans = 2
chans = 64
num_pool_layers = 6

# policy params
discount = .5

# tester params
fulldim = 144
base    = 8
budget  = 16
device  = torch.device("cuda:0")
save_dir = '/home/ec2-user/SageMaker/RLsamp/output/'

infostr = 'DDQN_test'

In [ ]:
ncfiles = np.load('/home/ec2-user/SageMaker/RLsamp/test_files.npz')['files']
loader  = ocmrLoader(ncfiles,batch_size=1,datapath=datapath,t_backtrack=t_backtrack,train_mode=False)

model   = poly_net(samp_dim=fulldim,in_chans=t_backtrack)
model.load_state_dict(hist['dqn_weights'])

unet_rand = Unet(in_chans=in_chans,out_chans=1,chans=chans,
            num_pool_layers=num_pool_layers,drop_prob=0).to(device)
rand_checkpoint = torch.load(unet_rand_dir)
unet_rand.load_state_dict(rand_checkpoint['model_state_dict'])

unet_lowfreq = Unet(in_chans=in_chans,out_chans=1,chans=chans,
            num_pool_layers=num_pool_layers,drop_prob=0).to(device)
lowfreq_checkpoint = torch.load(unet_lowfreq_dir)
unet_lowfreq.load_state_dict(lowfreq_checkpoint['model_state_dict'])

policy = DQN(model,[],device=device,gamma=discount,lr=0,
                  double_q_mode=True,unet=unet_lowfreq,mag_weight=5,maxGuideEp=0)


tester = DeepQL_tester(loader,policy,
                         eps=1e-3,
                         fulldim=fulldim,
                         base=base,
                         budget=budget,
                         save_dir=save_dir,
                         compare=True,
                         rand_eval_unet=unet_rand,
                         lowfreq_eval_unet=unet_lowfreq,
                         infostr=None,
                         device=device
                         )

In [ ]:
tester.test()

In [ ]:
eval_hist_path = '/home/ec2-user/SageMaker/RLsamp/output/EVAL_DQN_doubleQ_True_ba8_bu16_2023-05-08.pt'
eval_hist = torch.load(eval_hist_path)

In [ ]:
eval_hist

# Run Tester : REINFORCE

In [3]:
from RL_samp.REINFORCE import REINFORCE_tester
from unet.unet_model_fbr import Unet

In [4]:
hist_dir = '/home/ec2-user/SageMaker/RLsamp/output/REINFORCE_hist_2023-04-24_base8_budget16_BA8_BU16_E1000_G0_H1e-2_wTrue_magweg5_rwd1.pt'
hist = torch.load(hist_dir)

In [5]:
# dataloader params
t_backtrack = 8
datapath = '/home/ec2-user/SageMaker/data/OCMR_fully_sampled_images/'

# unet params
unet_rand_dir = '/home/ec2-user/SageMaker/RLsamp/output/recon_models/unet_lowfreq_rand_0.0_fbr_2_chans_64base8_budget16.pt'
unet_lowfreq_dir = '/home/ec2-user/SageMaker/RLsamp/output/recon_models/unet_lowfreq_rand_1.0_fbr_2_chans_64base8_budget16.pt'
in_chans = 2
chans = 64
num_pool_layers = 6

# policy params
discount = .5

# tester params
fulldim = 144
base    = 8
budget  = 16
device  = torch.device("cuda:0")
save_dir = '/home/ec2-user/SageMaker/RLsamp/output/'

infostr = 'REINFORCE_test'

In [6]:
ncfiles = np.load('/home/ec2-user/SageMaker/RLsamp/test_files.npz')['files']
loader  = ocmrLoader(ncfiles,batch_size=1,datapath=datapath,t_backtrack=t_backtrack,train_mode=False)

model   = poly_net(samp_dim=fulldim,softmax=True,in_chans=t_backtrack)
model.load_state_dict(hist['polynet_weights'])

unet_rand = Unet(in_chans=in_chans,out_chans=1,chans=chans,
            num_pool_layers=num_pool_layers,drop_prob=0).to(device)
rand_checkpoint = torch.load(unet_rand_dir)
unet_rand.load_state_dict(rand_checkpoint['model_state_dict'])

unet_lowfreq = Unet(in_chans=in_chans,out_chans=1,chans=chans,
            num_pool_layers=num_pool_layers,drop_prob=0).to(device)
lowfreq_checkpoint = torch.load(unet_lowfreq_dir)
unet_lowfreq.load_state_dict(lowfreq_checkpoint['model_state_dict'])

<All keys matched successfully>

In [7]:
tester = REINFORCE_tester(loader,model,
                         fulldim=fulldim,
                         base=base,
                         budget=budget,
                         save_dir=save_dir,
                         unet=unet_lowfreq,
                         rand_eval_unet=unet_rand,
                         lowfreq_eval_unet=unet_lowfreq,
                         infostr=infostr,
                         device=device
                         )

current file: fs_0035_3T.pt
Dimension of the current data file: t_ubd 18, slice_ubd 1, rep_ubd 1


In [8]:
tester.run()

file [1/10] rep [1/1] slice [1/1]
current file: fs_0021_3T.pt
Dimension of the current data file: t_ubd 22, slice_ubd 1, rep_ubd 1
file [2/10] rep [1/1] slice [1/1]
current file: fs_0074_1_5T.pt
Dimension of the current data file: t_ubd 19, slice_ubd 12, rep_ubd 1
file [3/10] rep [1/1] slice [1/12]
current file: fs_0038_3T.pt
Dimension of the current data file: t_ubd 16, slice_ubd 1, rep_ubd 1
file [4/10] rep [1/1] slice [1/1]
current file: fs_0023_3T.pt
Dimension of the current data file: t_ubd 22, slice_ubd 1, rep_ubd 1
file [5/10] rep [1/1] slice [1/1]
current file: fs_0025_3T.pt
Dimension of the current data file: t_ubd 27, slice_ubd 1, rep_ubd 1
file [6/10] rep [1/1] slice [1/1]
current file: fs_0060_1_5T.pt
Dimension of the current data file: t_ubd 21, slice_ubd 12, rep_ubd 1
file [7/10] rep [1/1] slice [1/12]
current file: fs_0041_3T.pt
Dimension of the current data file: t_ubd 18, slice_ubd 1, rep_ubd 1
file [8/10] rep [1/1] slice [1/1]
current file: fs_0042_3T.pt
Dimension of 